In [37]:
import qiskit
import numpy as np
from qiskit import *
from qiskit.visualization import plot_histogram, plot_bloch_vector, plot_bloch_multivector
from qiskit.tools.monitor import job_monitor
from numpy.random import randint

In [38]:
def bin_to_int_key(bin_key):
    n = len(bin_key)
    int_key = []
    zero = []
    one = []
    for i in range(0,n):
        if not bin_key[i]:
            zero.append(i)
        else:
            one.append(i)
    int_key = zero + one
    return int_key

In [39]:
def chained_cnot(n, KAT, qc):
    # n = len(P)
    for i in range(0,n):
        if i != KAT[i]:
            qc.cnot(i, KAT[i])

In [40]:
def chained_cnot_dec(n, KAT, qc):
    # n = len(P)
    for i in reversed(range(0,n)):
        if i != KAT[i]:
            qc.cnot(i, KAT[i])

In [41]:
def qotp(n, KAT_2n, qc):
    # n = len(P)
    # KAT has length 2n
    for i in range(0,n):
        if KAT_2n[2*i]:
            qc.x(i)
        if KAT_2n[2*i+1]:
            qc.z(i)

In [ ]:
def qotp_dec(n, KAT_2n, qc):
    # n = len(P)
    # KAT has length 2n
    for i in range(0,n):
        if KAT_2n[2*i+1]:
            qc.z(i)
        if KAT_2n[2*i]:
            qc.x(i)

In [42]:
def random_init(message, qc):
    for i in range(0, len(message)):
        if message[i]:
            qc.x(i)

In [43]:
n = 10
np.random.seed(seed = 2)
alice_trent_shared_bin = randint(2, size = n) # alice's message P
KAT = bin_to_int_key(alice_trent_shared_bin)
KAT_2n = randint(2, size = 2*n)
message = randint(2, size = n)

In [44]:
print(alice_trent_shared_bin)
print(KAT)
print(KAT_2n)
print(message)

[0 1 1 0 0 1 0 1 0 1]
[0, 3, 4, 6, 8, 1, 2, 5, 7, 9]
[0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 0 0 1 1]
[1 0 0 1 0 0 1 1 1 0]


In [45]:
q = QuantumRegister(n)
c = ClassicalRegister(n)
qc = QuantumCircuit(q, c)
random_init(message, qc)
qc.barrier()
# use either encryption method
# chained_cnot(n, KAT, qc)
qotp(n, KAT_2n, qc)
qc.barrier()
qc.draw()

┌───┐ ░ ┌───┐      ░ 
q78_0: ┤ X ├─░─┤ Z ├──────░─
       └───┘ ░ ├───┤┌───┐ ░ 
q78_1: ──────░─┤ X ├┤ Z ├─░─
             ░ ├───┤├───┤ ░ 
q78_2: ──────░─┤ X ├┤ Z ├─░─
       ┌───┐ ░ ├───┤├───┤ ░ 
q78_3: ┤ X ├─░─┤ X ├┤ Z ├─░─
       └───┘ ░ └───┘└───┘ ░ 
q78_4: ──────░────────────░─
             ░            ░ 
q78_5: ──────░────────────░─
       ┌───┐ ░ ┌───┐┌───┐ ░ 
q78_6: ┤ X ├─░─┤ X ├┤ Z ├─░─
       ├───┤ ░ ├───┤└───┘ ░ 
q78_7: ┤ X ├─░─┤ X ├──────░─
       ├───┤ ░ └───┘      ░ 
q78_8: ┤ X ├─░────────────░─
       └───┘ ░ ┌───┐┌───┐ ░ 
q78_9: ──────░─┤ X ├┤ Z ├─░─
             ░ └───┘└───┘ ░ 
c3: 10/═════════════════════

In [46]:
# chained_cnot_dec(n, KAT, qc)
qotp(n, KAT_2n, qc)
qc.draw()

┌───┐ ░ ┌───┐      ░ ┌───┐     
q78_0: ┤ X ├─░─┤ Z ├──────░─┤ Z ├─────
       └───┘ ░ ├───┤┌───┐ ░ ├───┤┌───┐
q78_1: ──────░─┤ X ├┤ Z ├─░─┤ X ├┤ Z ├
             ░ ├───┤├───┤ ░ ├───┤├───┤
q78_2: ──────░─┤ X ├┤ Z ├─░─┤ X ├┤ Z ├
       ┌───┐ ░ ├───┤├───┤ ░ ├───┤├───┤
q78_3: ┤ X ├─░─┤ X ├┤ Z ├─░─┤ X ├┤ Z ├
       └───┘ ░ └───┘└───┘ ░ └───┘└───┘
q78_4: ──────░────────────░───────────
             ░            ░           
q78_5: ──────░────────────░───────────
       ┌───┐ ░ ┌───┐┌───┐ ░ ┌───┐┌───┐
q78_6: ┤ X ├─░─┤ X ├┤ Z ├─░─┤ X ├┤ Z ├
       ├───┤ ░ ├───┤└───┘ ░ ├───┤└───┘
q78_7: ┤ X ├─░─┤ X ├──────░─┤ X ├─────
       ├───┤ ░ └───┘      ░ └───┘     
q78_8: ┤ X ├─░────────────░───────────
       └───┘ ░ ┌───┐┌───┐ ░ ┌───┐┌───┐
q78_9: ──────░─┤ X ├┤ Z ├─░─┤ X ├┤ Z ├
             ░ └───┘└───┘ ░ └───┘└───┘
c3: 10/═══════════════════════════════

In [47]:
def get_measurements(measure_type, qc, num_shots): 
    qasm_sim = Aer.get_backend('qasm_simulator')
    if measure_type == "simulator":
        job = execute(qc, qasm_sim, memory = True, shots = num_shots) 
        result = job.result()
        return result
    elif measure_type == "real_device":
        job = execute(qc, backend = backend, memory = True, shots = num_shots) 
        job_monitor(job)
        result = job.result()
        return result

In [48]:
qc.measure(q,c)
res = get_measurements("simulator", qc, 1024)
counts = res.get_counts()
print(counts)
print(message)

{'0111001001': 1024}
[1 0 0 1 0 0 1 1 1 0]
